In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [3]:
from datetime import datetime
import os

import numpy as np
import pandas as pd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADBE', 'ADYEY', 'AMD', 'AMZN', 'ANET',
    'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'DDOG', 'DOCS', 'DOCU',
    'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH', 'GMED', 'GOOG', 'INTG',
    'ISRG', 'JD', 'LSPD', 'LULU', 'MDB', 'MELI', 'MNST', 'MSFT', 'MTCH',
    'NVCR', 'NVDA', 'OKTA', 'OPEN', 'PANW', 'PAYC', 'PINS', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'RIVN', 'ROKU', 'RVLV', 'SE', 'SHOP',
    'SNOW', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO',
    'TWTR', 'UPST', 'VEEV', 'VLD', 'XPEV', 'ZEN', 'ZM']

In [5]:
!ls $DL

In [6]:
#!head -5 $DL/Positions.csv

In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [9]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADBE
ADYEY


### E*Trade

In [17]:
DONGMEI = False

In [18]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,661.4
ABMD,583
ABNB,351.33
ADYEY,5567.6
AMD,2250.82


In [19]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,661
ABMD,583
ABNB,351
ADYEY,5568
AMD,2251


In [20]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [25]:
et_df.loc[et_df.ET == '--']
#et_df.loc['DUOL', 'ET'] = 103.07

,ET


In [26]:
check_symbols(et_df)

All clear


In [27]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,661
ABMD,583
ABNB,351
ADBE,0
ADYEY,5568


### Fidelity

In [28]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Aug-05-2022.csv'

In [29]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Aug-05-2022.csv
Positions.csv
mw.csv


In [30]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("06", "07")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,10746.66,$1.00,$0.00,$10746.66,NaN,NaN,NaN,NaN,26.33%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$291.50,-$0.74,$1166.00,-$2.96,-0.26%,+$147.31,+14.46%,2.86%,$1018.69,$254.67,Cash
2,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,2.00,$117.11,-$2.11,$234.22,-$4.22,-1.77%,-$51.28,-17.97%,0.57%,$285.50,$142.75,Cash
3,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,16.00,$102.31,-$1.60,$1636.96,-$25.60,-1.54%,+$51.77,+3.26%,4.01%,$1585.19,$99.07,Cash
4,218520300,ROLLOVER IRA,AXON,AXON ENTERPRISE INC COM,2.00,$116.29,-$0.31,$232.58,-$0.62,-0.27%,-$15.54,-6.27%,0.57%,$248.12,$124.06,Cash


In [31]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,ABMD,1166
2,ROLLOVER IRA,ABNB,234
3,ROLLOVER IRA,AMD,1637
4,ROLLOVER IRA,AXON,233
5,ROLLOVER IRA,BROS,442


In [32]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
ABMD,1166
ABNB,234
AMD,1637
AXON,233
BROS,442


In [33]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [34]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,661,0,8433,0
ABMD,583,1166,2332,0
ABNB,351,234,351,0
ADBE,0,0,0,0
ADYEY,5568,0,3161,0


In [35]:
out.to_csv(f'{DL}/clean.csv')

In [36]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


,Value_1,Value_2,Value_3
AAPL,827,2480,1654
ABMD,1166,874,0
ABNB,703,586,1171
ADBE,0,0,0
ADYEY,736,2101,1598


In [12]:
check_symbols(mw)

APPH not in SYMBOLS
COIN not in SYMBOLS
COUR not in SYMBOLS
CRWD not in SYMBOLS
DKNG not in SYMBOLS
FUV not in SYMBOLS
FVRR not in SYMBOLS
NYT not in SYMBOLS
PTON not in SYMBOLS
SPLK not in SYMBOLS
SQ not in SYMBOLS
WK not in SYMBOLS


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,827,2480,1654
ABMD,1166,874,0
ABNB,703,586,1171
ADBE,0,0,0
ADYEY,736,2101,1598


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [15]:
!open $DL/mw.csv

In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*